해당 코드들은 자동화 최적화가 되어 있지 않음

아직은 1개씩만 돌려보고 결과를 확인하기 위한 코드입니다.

In [1]:
from collections import deque
import os
import numpy as np
import pandas as pd
import json
import cv2
import mediapipe as mp
import time

In [6]:
video_file_paths, video_file_names = [], []
json_paths, json_names = [], []
for root, dirs, files in os.walk("."):
    for fname in files:
        if fname.endswith("rgb_face.mp4"):
            video_file_paths.append(os.path.join(root, fname))
            video_file_names.append(fname)
        if fname.endswith("drowsiness.json"):
            json_paths.append(os.path.join(root, fname))
            json_names.append(fname)

print(video_file_names)
print(json_names)
print(video_file_paths)
print(json_paths)

['gA_1_s5_2019-03-14T14;26;17+01;00_rgb_face.mp4', 'gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face.mp4', 'gA_3_s5_2019-03-13T09;36;25+01;00_rgb_face.mp4', 'gA_4_s5_2019-03-13T10;56;52+01;00_rgb_face.mp4', 'gA_5_s5_2019-03-13T09;06;49+01;00_rgb_face.mp4', 'gB_10_s5_2019-03-12T10;35;20+01;00_rgb_face.mp4', 'gB_6_s5_2019-03-13T13;37;11+01;00_rgb_face.mp4', 'gB_7_s5_2019-03-13T13;55;52+01;00_rgb_face.mp4', 'gB_8_s5_2019-03-13T14;10;09+01;00_rgb_face.mp4', 'gB_9_s5_2019-03-07T16;31;48+01;00_rgb_face.mp4', 'gC_11_s5_2019-03-12T09;08;15+01;00_rgb_face.mp4', 'gC_12_s5_2019-03-14T09;56;52+01;00_rgb_face.mp4', 'gC_13_s5_2019-03-12T10;03;00+01;00_rgb_face.mp4', 'gC_14_s5_2019-03-12T09;18;58+01;00_rgb_face.mp4', 'gC_15_s5_2019-03-12T11;03;23+01;00_rgb_face.mp4']
['gA_1_s5_2019-03-14T14;26;17+01;00_rgb_ann_drowsiness.json', 'gA_2_s5_2019-03-13T09;19;23+01;00_rgb_ann_drowsiness.json', 'gA_3_s5_2019-03-13T09;36;25+01;00_rgb_ann_drowsiness.json', 'gA_4_s5_2019-03-13T10;56;52+01;00_rgb_ann_drowsiness.json'

# Mediapipe 영상 labeling

In [3]:
CALIB_SECONDS = 2.0       # 초기 캘리브레이션 구간(초)
FPS_FALLBACK = 30.0       # FPS 정보가 없을 때 기본값
SMOOTH_WIN = 5            # EAR/MAR 이동평균 윈도우
BLINK_MAX_FRAMES = 8      # blink로 볼 수 있는 최대 닫힘 프레임 길이
YAWN_MIN_FRAMES = 30      # 하품으로 간주할 최소 프레임 길이
HAND_MOUTH_DIST_PX = 80   # 손가락 포인트와 입 중심 간 근접 판정 거리(픽셀)

# 상태 머신 임계치 비율 (캘리브레이션 결과에 곱해 사용)
EYE_CLOSE_RATIO = 0.85    # 눈감김 임계치: EAR_low = median(EAR_calib)*EYE_CLOSE_RATIO
EYE_OPEN_RATIO  = 1.05    # 눈뜸 임계치: EAR_high = median(EAR_calib)*EYE_OPEN_RATIO
MOUTH_YAWN_RATIO = 1.25   # 하품 임계치: MAR_high = median(MAR_calib)*MOUTH_YAWN_RATIO

# 졸림 지표 윈도우(초)
PERCLOS_WIN_SEC = 10.0
RATE_WIN_SEC = 20.0

# 졸림 임계값(튜닝 가능)
PERCLOS_T1, PERCLOS_T2, PERCLOS_T3 = 0.20, 0.40, 0.60
YAWN_T1, YAWN_T2, YAWN_T3 = 2.0, 4.0, 6.0           # per minute
BLINK_DUR_T1, BLINK_DUR_T2, BLINK_DUR_T3 = 0.25, 0.35, 0.50  # seconds
LONG_EC_T1, LONG_EC_T2, LONG_EC_T3 = 0.5, 0.8, 1.0            # seconds

In [4]:
# FaceMesh / Hands 초기화
mp_face = mp.solutions.face_mesh
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# FaceMesh 눈/입 계산용 랜드마크 인덱스 (MediaPipe FaceMesh)
# EAR: (상하 거리 합) / (좌우 거리)  -- 관례적 정의
LEFT_EYE = [33, 160, 158, 133, 153, 144]   # [left, top1, top2, right, bottom1, bottom2]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
# MAR: (상하 거리) / (좌우 거리)
MOUTH_HORZ = (61, 291)    # 좌우 외측 입꼬리
MOUTH_VERT = (13, 14)     # 상하(안쪽 입술 중앙)

# 상태 관련 버퍼
ear_buf = deque(maxlen=SMOOTH_WIN)
mar_buf = deque(maxlen=SMOOTH_WIN)

# 상태 머신 상수 (눈 + 하품)
EYE_OPEN, EYE_CLOSE, EYE_OPENING, EYE_CLOSING, EYE_BLINK = range(5)
YAWN_NONE, YAWN_WITH_HAND, YAWN_WITHOUT_HAND = 0, 1, 2

eye_state = EYE_OPEN    # 초기 가정
close_count = 0         # 연속 닫힘 프레임 수(블링크 판정용)
yawn_state = YAWN_NONE
yawn_count = 0

# 캘리브레이션 샘플
EAR_LOW, EAR_HIGH = 0.18, 0.26
MAR_HIGH = 0.60
ear_samples = []
mar_samples = []

# 졸림 지표 계산용 슬라이딩 버퍼 (초 단위 시간축)
closed_flags = deque()   # (t, 0/1) — 눈감김 여부
blink_events = deque()   # (t, duration_sec)
yawn_events  = deque()   # (t,)
active_close_start = None  # 현재 진행 중인 eye-closure 시작시간
active_blink_start = None  # blink용(짧은 닫힘)

# 졸음 단계 색상 팔레트
LEVEL_COLOR = {
    0: (0, 220, 0),     # alert - green
    1: (0, 200, 255),   # mild - orange
    2: (0, 128, 255),   # moderate - darker orange
    3: (0, 0, 255)      # severe - red
}

In [5]:
# 각종 상태 및 라벨을 판별하는 함수들 + ROI
def euclid(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

def eye_aspect_ratio(lm, eye_idx, w, h):
    pts = [(lm[i].x*w, lm[i].y*h) for i in eye_idx]
    p1, p2, p3, p4, p5, p6 = pts
    vertical = (euclid(p2, p6)+euclid(p3, p5))/2.0
    horizontal = euclid(p1, p4)+1e-6
    return vertical / horizontal

def mouth_aspect_ratio(lm, w, h):
    L = (lm[MOUTH_HORZ[0]].x*w, lm[MOUTH_HORZ[0]].y*h)
    R = (lm[MOUTH_HORZ[1]].x*w, lm[MOUTH_HORZ[1]].y*h)
    U = (lm[MOUTH_VERT[0]].x*w, lm[MOUTH_VERT[0]].y*h)
    D = (lm[MOUTH_VERT[1]].x*w, lm[MOUTH_VERT[1]].y*h)
    horizontal = euclid(L, R) + 1e-6
    vertical = euclid(U, D)
    return vertical / horizontal, ((L[0]+R[0])/2, (U[1]+D[1])/2)

def moving_avg(buf, k):
    if len(buf)==0: return None
    return float(np.mean(list(buf)[-k:]))

def draw_roi(frame, pts, color=(0,255,0), thickness=2):
    xs = [p[0] for p in pts]; ys = [p[1] for p in pts]
    x1, y1, x2, y2 = int(min(xs)), int(min(ys)), int(max(xs)), int(max(ys))
    cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
    return (x1, y1, x2, y2)

def is_hand_near_mouth(hand_lm_list, mouth_center, max_dist_px, w, h):
    if hand_lm_list is None: return False
    cx, cy = mouth_center
    for hand in hand_lm_list:
        for lm in hand.landmark:
            px, py = lm.x*w, lm.y*h
            if euclid((px, py), (cx, cy)) <= max_dist_px:
                return True
    return False

def drowsiness_level(perclos, yawn_rate, avg_blink_dur, longest_ec):
    # 가장 심한 상태 조건을 우선 판단
    # Severe 우선
    if (perclos is not None and perclos > PERCLOS_T3) or \
       (yawn_rate is not None and yawn_rate > YAWN_T3) or \
       (longest_ec is not None and longest_ec >= LONG_EC_T3):
        return 3, "severe_drowsy"
    # Moderate
    if (perclos is not None and perclos > PERCLOS_T2) or \
       (yawn_rate is not None and yawn_rate > YAWN_T2) or \
       (avg_blink_dur is not None and avg_blink_dur > BLINK_DUR_T2) or \
       (longest_ec is not None and longest_ec >= LONG_EC_T2):
        return 2, "moderate_drowsy"
    # Mild
    if (perclos is not None and perclos > PERCLOS_T1) or \
       (yawn_rate is not None and yawn_rate > YAWN_T1) or \
       (avg_blink_dur is not None and avg_blink_dur > BLINK_DUR_T1) or \
       (longest_ec is not None and longest_ec >= LONG_EC_T1):
        return 1, "mild_drowsy"
    # Alert
    return 0, "alert"

# 행동 라벨링 영상 추출 영역

In [ ]:
# MediaPipe 컨텍스트
# EAR 및 MAR 영역 표시 + 행동상태 라벨링(졸림상태는 X) 
start = time.time()
with mp_face.FaceMesh(
    static_image_mode=False,
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh, mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as hands:

    frame_idx = 0

    # 1) 캘리브레이션
    while True:
        ret, frame = cap.read()
        if not ret: 
              # print("Webcam frame read failed.")
              break
        frame_idx += 1
        t_sec = frame_idx / fps

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_res = face_mesh.process(rgb)
        hand_res = None 

        if face_res.multi_face_landmarks:
            face_landmarks = face_res.multi_face_landmarks[0].landmark
            ear_l = eye_aspect_ratio(face_landmarks, LEFT_EYE, width, height)
            ear_r = eye_aspect_ratio(face_landmarks, RIGHT_EYE, width, height)
            ear = (ear_l + ear_r) / 2.0
            mar, mouth_center = mouth_aspect_ratio(face_landmarks, width, height)
            ear_samples.append(ear)
            mar_samples.append(mar)

        if frame_idx >= calib_frames:
            break
    # 임계치 계산
    if len(ear_samples) >= 5:
        ear_med = float(np.median(ear_samples))
        EAR_LOW  = ear_med * EYE_CLOSE_RATIO
        EAR_HIGH = ear_med * EYE_OPEN_RATIO
    else:
        EAR_LOW, EAR_HIGH = 0.18, 0.26  # 안전 기본값 (얼굴 크기/거리 따라 다를 수 있음)

    if len(mar_samples) >= 5:
        mar_med = float(np.median(mar_samples))
        MAR_HIGH = mar_med * MOUTH_YAWN_RATIO
    else:
        MAR_HIGH = 0.5  # 안전 기본값
    

    # 2) 본 처리 루프 (다시 처음부터)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret: break
        frame_idx += 1

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_res = face_mesh.process(rgb)
        hands_res = hands.process(rgb)

        label_id = 0
        label_str = "eyes_state/open"

        if face_res.multi_face_landmarks:
            face_landmarks = face_res.multi_face_landmarks[0].landmark

            # EAR/MAR
            ear_l = eye_aspect_ratio(face_landmarks, LEFT_EYE, width, height)
            ear_r = eye_aspect_ratio(face_landmarks, RIGHT_EYE, width, height)
            ear = (ear_l + ear_r) / 2.0
            mar, mouth_center = mouth_aspect_ratio(face_landmarks, width, height)

            ear_buf.append(ear)
            mar_buf.append(mar)
            ear_s = moving_avg(ear_buf, SMOOTH_WIN)
            mar_s = moving_avg(mar_buf, SMOOTH_WIN)

            # 눈 상태 머신 업데이트
            prev_state = eye_state
            if ear_s is None:
                eye_state = EYE_OPEN
            else:
                # opening/closing은 EAR 변화율로 판정(최근 3프레임) <- 5프레임
                if len(ear_buf) >= 3:
                    ear_deriv = ear_buf[-1] - ear_buf[-3]
                else:
                    ear_deriv = 0.0

                is_closed = ear_s < EAR_LOW
                is_opened = ear_s > EAR_HIGH

                if is_closed:
                    eye_state = EYE_CLOSE if prev_state == EYE_CLOSE else (EYE_CLOSING if ear_deriv < 0 else EYE_CLOSE)
                    close_count += 1
                elif is_opened:
                    eye_state = EYE_OPEN if prev_state == EYE_OPEN else (EYE_OPENING if ear_deriv > 0 else EYE_OPEN)
                    if 0 < close_count <= BLINK_MAX_FRAMES:
                        eye_state = EYE_BLINK
                    close_count = 0
                else:
                    eye_state = EYE_OPENING if ear_deriv > 0 else (EYE_CLOSING if ear_deriv < 0 else prev_state)
                    if prev_state in (EYE_CLOSE, EYE_CLOSING):
                        close_count += 1
                    else:
                        close_count = 0

            # 하품 상태 판별
            hand_near = is_hand_near_mouth(
                hands_res.multi_hand_landmarks if hands_res else None,
                mouth_center, HAND_MOUTH_DIST_PX, width, height
            )
            if mar_s is not None and mar_s > MAR_HIGH:
                yawn_count += 1
                yawn_state = YAWN_WITH_HAND if hand_near else YAWN_WITHOUT_HAND
            else:
                yawn_state = YAWN_NONE
                yawn_count = 0

            # 최종 라벨 우선순위: 하품 > 눈 상태/눈 깜빡임
            if yawn_state != YAWN_NONE and yawn_count >= YAWN_MIN_FRAMES:
                if yawn_state == YAWN_WITH_HAND:
                    label_id, label_str = 5, "yawning/Yawning with hand"
                else:
                    label_id, label_str = 6, "yawning/Yawning without hand"
            else:
                if eye_state == EYE_OPEN: label_id, label_str = 0, "eyes_state/open"
                elif eye_state == EYE_CLOSE: label_id, label_str = 1, "eyes_state/close"
                elif eye_state == EYE_OPENING: label_id, label_str = 2, "eyes_state/opening"
                elif eye_state == EYE_CLOSING: label_id, label_str = 3, "eyes_state/closing"
                elif eye_state == EYE_BLINK:   label_id, label_str = 4, "blinks/blinking"

            # 얼굴 랜드마크 시각화 드로잉
            mp_draw.draw_landmarks(
                frame,
                face_res.multi_face_landmarks[0],
                mp_face.FACEMESH_TESSELATION,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_styles.get_default_face_mesh_tesselation_style()
            )
            # 눈/입 ROI 박스
            def idx_to_pts(idxs):
                return [(int(face_landmarks[i].x*width), int(face_landmarks[i].y*height)) for i in idxs]

            left_eye_pts = idx_to_pts(LEFT_EYE)
            right_eye_pts = idx_to_pts(RIGHT_EYE)
            mouth_pts = idx_to_pts([MOUTH_HORZ[0], MOUTH_HORZ[1], MOUTH_VERT[0], MOUTH_VERT[1]])
            draw_roi(frame, left_eye_pts, (0,255,0), 2)
            draw_roi(frame, right_eye_pts, (0,255,0), 2)
            draw_roi(frame, mouth_pts, (255,0,0), 2)

            # 손 랜드마크
            if hands_res and hands_res.multi_hand_landmarks:
                for hlm in hands_res.multi_hand_landmarks:
                    mp_draw.draw_landmarks(frame, hlm, mp_hands.HAND_CONNECTIONS)

            # 라벨/지표 텍스트
            cv2.putText(frame, f"Label {label_id}: {label_str}", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 200, 255), 2)
            if ear_s is not None:
                cv2.putText(frame, f"EAR:{ear_s:.3f}  (low:{EAR_LOW:.3f} high:{EAR_HIGH:.3f})",
                            (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            if mar_s is not None:
                cv2.putText(frame, f"MAR:{mar_s:.3f}  (yawn>{MAR_HIGH:.3f})",
                            (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 200, 0), 2)
            if yawn_state != YAWN_NONE:
                cv2.putText(frame, f"HandNearMouth: {bool(hand_near)}  YawnFrames:{yawn_count}",
                            (20, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 180, 255), 2)
        else:
            # 얼굴 미검출 시 기본 라벨 유지(열림 가정)
            label_id, label_str = 0, "eyes_state/open"
            cv2.putText(frame, "Face not detected", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

        # 프레임 로그
        logs.append({
            "frame": frame_idx,
            "label_id": label_id,
            "label_name": label_str,
            "EAR": float(ear_buf[-1]) if len(ear_buf)>0 else np.nan,
            "MAR": float(mar_buf[-1]) if len(mar_buf)>0 else np.nan
        })

        out.write(frame)


cap.release()
out.release()
cv2.destroyAllWindows()

pd.DataFrame(logs).to_csv("./output/per_frame_labels.csv", index=False)
print(f"Saved video to: {OUTPUT_VIDEO}")
print(f"Saved per-frame CSV to: {OUTPUT_CSV}")
print(f"{time.time()-start:.4f} sec")

32.0774 sec


# 졸음 상태 라벨링 영상 추출 영역

In [7]:
def drowsiness_labeling(cap, output, logs, fps, width, height):
    with mp_face.FaceMesh(
       static_image_mode=False,
       refine_landmarks=True,
       max_num_faces=1,
       min_detection_confidence=0.5,
       min_tracking_confidence=0.5
       ) as face_mesh, mp_hands.Hands(
       static_image_mode=False,
       max_num_hands=2,
       min_detection_confidence=0.5,
       min_tracking_confidence=0.5
    ) as hands:
       global EAR_HIGH, EAR_LOW, MAR_HIGH

       frame_idx = 0
       eye_state = EYE_OPEN    # 초기 가정
       close_count = 0         # 연속 닫힘 프레임 수(블링크 판정용)
       yawn_state = YAWN_NONE
       yawn_count = 0

       active_close_start = None  # 현재 진행 중인 eye-closure 시작시간
       active_blink_start = None  # blink용(짧은 닫힘)

       ear_buf.clear()
       mar_buf.clear()
       blink_events.clear()
       yawn_events.clear()
       closed_flags.clear()

       while True:
              ret, frame = cap.read()
              if not ret: break
              frame_idx += 1
              t_sec = frame_idx / fps

              rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
              face_res = face_mesh.process(rgb)
              hands_res = None  # 조건부로 돌려도 됨(여기선 항상 실행)

              label_id = 0
              label_str = "eyes_state/open"

              if face_res.multi_face_landmarks:
                     face_landmarks = face_res.multi_face_landmarks[0].landmark

                     # EAR/MAR
                     ear_l = eye_aspect_ratio(face_landmarks, LEFT_EYE, width, height)
                     ear_r = eye_aspect_ratio(face_landmarks, RIGHT_EYE, width, height)
                     ear = (ear_l + ear_r) / 2.0
                     mar, mouth_center = mouth_aspect_ratio(face_landmarks, width, height)

                     ear_buf.append(ear)
                     mar_buf.append(mar)
                     ear_s = moving_avg(ear_buf, SMOOTH_WIN)
                     mar_s = moving_avg(mar_buf, SMOOTH_WIN)

                     # 캘리브레이션 샘플 수집
                     if frame_idx <= int(CALIB_SECONDS * fps):
                            ear_samples.append(ear)
                            mar_samples.append(mar)
                            if frame_idx == int(CALIB_SECONDS * fps):
                                   if len(ear_samples) >= 5:
                                          ear_med = float(np.median(ear_samples))
                                          EAR_LOW  = ear_med * EYE_CLOSE_RATIO
                                          EAR_HIGH = ear_med * EYE_OPEN_RATIO
                                   if len(mar_samples) >= 5:
                                          mar_med = float(np.median(mar_samples))
                                          MAR_HIGH = mar_med * MOUTH_YAWN_RATIO

                     # 손-입 근접
                     hands_res = hands.process(rgb)
                     hand_near = is_hand_near_mouth(
                            hands_res.multi_hand_landmarks if hands_res else None,
                            mouth_center, HAND_MOUTH_DIST_PX, width, height
                     )

                     # 눈 상태 머신
                     prev_state = eye_state
                     if ear_s is None:
                            eye_state = EYE_OPEN
                     else:
                            if len(ear_buf) >= 3:
                                   ear_deriv = ear_buf[-1] - ear_buf[-3]
                            else:
                                   ear_deriv = 0.0

                            is_closed = ear_s < EAR_LOW
                            is_opened = ear_s > EAR_HIGH

                            if is_closed:
                                   eye_state = EYE_CLOSE if prev_state == EYE_CLOSE else (EYE_CLOSING if ear_deriv < 0 else EYE_CLOSE)
                                   close_count += 1
                            elif is_opened:
                                   eye_state = EYE_OPEN if prev_state == EYE_OPEN else (EYE_OPENING if ear_deriv > 0 else EYE_OPEN)
                                   # blink 판정
                                   if 0 < close_count <= BLINK_MAX_FRAMES:
                                          eye_state = EYE_BLINK
                                   close_count = 0
                            else:
                                   eye_state = EYE_OPENING if ear_deriv > 0 else (EYE_CLOSING if ear_deriv < 0 else prev_state)
                                   if prev_state in (EYE_CLOSE, EYE_CLOSING):
                                          close_count += 1
                                   else:
                                          close_count = 0

                     # 하품 상태
                     if mar_s is not None and mar_s > MAR_HIGH:
                            yawn_count += 1
                            yawn_state = YAWN_WITH_HAND if hand_near else YAWN_WITHOUT_HAND
                     else:
                            yawn_state = YAWN_NONE
                            yawn_count = 0

                     # 최종 행동 라벨
                     if yawn_state != YAWN_NONE and yawn_count >= YAWN_MIN_FRAMES:
                            if yawn_state == YAWN_WITH_HAND:
                                   label_id, label_str = 5, "yawning/Yawning with hand"
                            else:
                                   label_id, label_str = 6, "yawning/Yawning without hand"
                     else:
                            if   eye_state == EYE_OPEN:    label_id, label_str = 0, "eyes_state/open"
                            elif eye_state == EYE_CLOSE:   label_id, label_str = 1, "eyes_state/close"
                            elif eye_state == EYE_OPENING: label_id, label_str = 2, "eyes_state/opening"
                            elif eye_state == EYE_CLOSING: label_id, label_str = 3, "eyes_state/closing"
                            elif eye_state == EYE_BLINK:   label_id, label_str = 4, "blinks/blinking"

                     # ====== 졸림 지표 업데이트 ======
                     # A) PERCLOS: 최근 PERCLOS_WIN_SEC 동안 '눈감김' 비율
                     closed_flag = 1 if ear_s is not None and ear_s < EAR_LOW else 0
                     closed_flags.append((t_sec, closed_flag))
                     while closed_flags and (t_sec - closed_flags[0][0] > PERCLOS_WIN_SEC):
                            closed_flags.popleft()
                     if closed_flags:
                            perclos = sum(f for _, f in closed_flags) / float(len(closed_flags))
                     else:
                            perclos = None

                     # B) blink 이벤트 기록(짧은 닫힘)
                     # blink가 찍히는 순간(eye_state == EYE_BLINK)에서 duration 계산
                     # 간단히: 방금 전까지의 close_count를 duration으로 사용
                     if eye_state == EYE_BLINK:
                            blink_dur = close_count / fps  # 방금 열림과 함께 close_count가 0으로 초기화되기 전에 계산됨
                            blink_events.append((t_sec, blink_dur))
                     # 창구 유지
                     while blink_events and (t_sec - blink_events[0][0] > RATE_WIN_SEC):
                            blink_events.popleft()

                     if blink_events:
                            blink_rate = len(blink_events) * (60.0 / RATE_WIN_SEC)  # per 20s
                            avg_blink_dur = float(np.mean([d for _, d in blink_events]))
                     else:
                            blink_rate = 0.0
                            avg_blink_dur = None

                     # C) 긴 eye-closure 감지(연속 close가 길면 이벤트로 기록)
                     # close 연속 구간의 시작/끝 추적
                     if closed_flag == 1 and active_close_start is None:
                            active_close_start = t_sec
                     if closed_flag == 0 and active_close_start is not None:
                            dur = t_sec - active_close_start
                            blink_events.append((t_sec, dur))  # 긴 eye closure도 blink_events에 포함시켜 평균/최대에 반영
                            active_close_start = None
                     # longest eye closure (최근 RATE_WIN_SEC)
                     if blink_events:
                            longest_ec = max(d for _, d in blink_events)
                     else:
                            longest_ec = None

                     # D) yawn 이벤트(프레임 지속 충족 시 시점 기록)
                     if yawn_state != YAWN_NONE and yawn_count == YAWN_MIN_FRAMES:
                            yawn_events.append((t_sec,))
                     while yawn_events and (t_sec - yawn_events[0][0] > RATE_WIN_SEC):
                            yawn_events.popleft()
                     yawn_rate = len(yawn_events) * (60.0 / RATE_WIN_SEC) if yawn_events else 0.0

                     # ====== 졸림 단계 산출 ======
                     d_level, d_label = drowsiness_level(perclos, yawn_rate, avg_blink_dur, longest_ec)

                     # ====== 시각화 ======
                     cv2.putText(frame, f"Label {label_id}: {label_str}", (20, 40),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 200, 255), 2)
                     if ear_s is not None:
                            cv2.putText(frame, f"EAR:{ear_s:.3f} (L:{EAR_LOW:.3f} H:{EAR_HIGH:.3f})",
                                          (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                     if mar_s is not None:
                            cv2.putText(frame, f"MAR:{mar_s:.3f} (Y>{MAR_HIGH:.3f})",
                                          (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,200,0), 2)

                     # 졸림 지표/단계 표시
                     c = LEVEL_COLOR[d_level]
                     cv2.putText(frame, f"Drowsiness {d_level}: {d_label}", (20, 135),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, c, 2)
                     cv2.putText(frame, f"PERCLOS:{(perclos if perclos is not None else np.nan):.2f}  "
                                          f"Yawn/min:{yawn_rate:.1f}  "
                                          f"Blink/min:{blink_rate:.1f}  "
                                          f"AvgBlinkDur:{(avg_blink_dur if avg_blink_dur else np.nan):.2f}s  "
                                          f"LongestEC:{(longest_ec if longest_ec else np.nan):.2f}s",
                                   (20, 165), cv2.FONT_HERSHEY_SIMPLEX, 0.3, c, 2)

              else:
                     # 얼굴 미검출 시
                     label_id, label_str = 0, "eyes_state/open"
                     d_level, d_label = 0, "alert"
                     perclos = None; yawn_rate = 0.0; blink_rate = 0.0; avg_blink_dur = None; longest_ec = None
                     cv2.putText(frame, "Face not detected", (20, 40),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 2)

              # ====== 로그 저장 ======
              logs.append({
                     "frame": frame_idx,
                     "time_sec": t_sec,
                     "label_id": label_id,
                     "label_name": label_str,
                     "EAR": float(ear_buf[-1]) if len(ear_buf)>0 else np.nan,
                     "MAR": float(mar_buf[-1]) if len(mar_buf)>0 else np.nan,
                     "yawn_rate_per_min": float(yawn_rate),
                     "blink_rate_per_min": float(blink_rate) if face_res.multi_face_landmarks else 0.0,
                     "avg_blink_dur_sec": float(avg_blink_dur) if avg_blink_dur is not None else np.nan,
                     "longest_eye_closure_sec": float(longest_ec) if longest_ec is not None else np.nan,
                     "drowsiness_level": d_level,
                     "drowsiness_label": d_label
              })

              output.write(frame)
    return logs

In [9]:
for video_path, video_name in zip(video_file_paths, video_file_names):
       cap = cv2.VideoCapture(video_path)

       # 컴퓨터 연결 카메라 사용, 실시간 웹캠 등
       #cap = cv2.VideoCapture(CAM_INDEX)   

       fps = cap.get(cv2.CAP_PROP_FPS)
       if fps <= 1e-2: fps = FPS_FALLBACK
       width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
       height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
       total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.get(cv2.CAP_PROP_FRAME_COUNT)>0 else None

       fourcc = cv2.VideoWriter_fourcc(*"mp4v")
       output_name = video_name.replace(".mp4", "-labeled.mp4")
       out = cv2.VideoWriter("./output/"+output_name, fourcc, fps, (width, height))

       logs = []

       # 캘리브 구간 프레임 수
       calib_frames = int(CALIB_SECONDS * fps)

       output_log = drowsiness_labeling(cap, out, logs, fps, width, height)

       cap.release()
       out.release()
       cv2.destroyAllWindows()

       pd.DataFrame(logs).to_csv("./output/"+output_name.replace(".mp4", ".csv"), index=False)
       print(f"Saved video: {"./output/"+output_name}")
       print(f"Saved CSV:   {"./output/"+output_name.replace(".mp4", ".csv")}")

Saved video: ./output/gA_1_s5_2019-03-14T14;26;17+01;00_rgb_face-labeled.mp4
Saved CSV:   ./output/gA_1_s5_2019-03-14T14;26;17+01;00_rgb_face-labeled.csv
Saved video: ./output/gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face-labeled.mp4
Saved CSV:   ./output/gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face-labeled.csv
Saved video: ./output/gA_3_s5_2019-03-13T09;36;25+01;00_rgb_face-labeled.mp4
Saved CSV:   ./output/gA_3_s5_2019-03-13T09;36;25+01;00_rgb_face-labeled.csv
Saved video: ./output/gA_4_s5_2019-03-13T10;56;52+01;00_rgb_face-labeled.mp4
Saved CSV:   ./output/gA_4_s5_2019-03-13T10;56;52+01;00_rgb_face-labeled.csv
Saved video: ./output/gA_5_s5_2019-03-13T09;06;49+01;00_rgb_face-labeled.mp4
Saved CSV:   ./output/gA_5_s5_2019-03-13T09;06;49+01;00_rgb_face-labeled.csv
Saved video: ./output/gB_10_s5_2019-03-12T10;35;20+01;00_rgb_face-labeled.mp4
Saved CSV:   ./output/gB_10_s5_2019-03-12T10;35;20+01;00_rgb_face-labeled.csv
Saved video: ./output/gB_6_s5_2019-03-13T13;37;11+01;00_rgb_face-labeled.m

In [11]:
csv_file_paths, csv_file_names = [], []
for root, dirs, files in os.walk("./output/"):
    for fname in files:
        if fname.endswith("-labeled.csv"):
            csv_file_paths.append(os.path.join(root, fname))
            csv_file_names.append(fname)
            
print(csv_file_names)
print(csv_file_paths)

['gA_1_s5_2019-03-14T14;26;17+01;00_rgb_face-labeled.csv', 'gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face-labeled.csv', 'gA_3_s5_2019-03-13T09;36;25+01;00_rgb_face-labeled.csv', 'gA_4_s5_2019-03-13T10;56;52+01;00_rgb_face-labeled.csv', 'gA_5_s5_2019-03-13T09;06;49+01;00_rgb_face-labeled.csv', 'gB_10_s5_2019-03-12T10;35;20+01;00_rgb_face-labeled.csv', 'gB_6_s5_2019-03-13T13;37;11+01;00_rgb_face-labeled.csv', 'gB_7_s5_2019-03-13T13;55;52+01;00_rgb_face-labeled.csv', 'gB_8_s5_2019-03-13T14;10;09+01;00_rgb_face-labeled.csv', 'gB_9_s5_2019-03-07T16;31;48+01;00_rgb_face-labeled.csv', 'gC_11_s5_2019-03-12T09;08;15+01;00_rgb_face-labeled.csv', 'gC_12_s5_2019-03-14T09;56;52+01;00_rgb_face-labeled.csv', 'gC_13_s5_2019-03-12T10;03;00+01;00_rgb_face-labeled.csv', 'gC_14_s5_2019-03-12T09;18;58+01;00_rgb_face-labeled.csv', 'gC_15_s5_2019-03-12T11;03;23+01;00_rgb_face-labeled.csv']
['./output/gA_1_s5_2019-03-14T14;26;17+01;00_rgb_face-labeled.csv', './output/gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face-la

In [12]:
# segment화 json 파일 생성 부분
# csv 파일을 읽은 후 → drowsiness level에 따라 구역을 segment화 시켜 각 구간의 길이를 계산
def make_segments(df, frame_col, label_col, time_col, insert_gaps=True):
    d = df.sort_values(by=frame_col).reset_index(drop=True).copy()
    d[frame_col] = d[frame_col].astype(int)
    d[time_col] = round(d[time_col].astype(float), 3)
    d[label_col] = d[label_col].astype(int)
    if d.empty: 
        return []

    segs = []
    start = int(d.loc[0, frame_col])
    start_time = float(d.loc[0, time_col])
    prev_frame = start
    prev_label = int(d.loc[0, label_col])

    for i in range(1, len(d)):
        f = int(d.loc[i, frame_col])
        t = float(d.loc[i, time_col])
        lab = int(d.loc[i, label_col])
        # 프레임 갭(누락 프레임) 처리
        if f != prev_frame + 1:
            segs.append({"start": start, "end": prev_frame + 1, "label": prev_label, "note": "프레임 누락"})
            if insert_gaps and f > prev_frame + 1:
                segs.append({"start": prev_frame + 1, "end": f, "label": -1, "note": "무시"})  # 전이/무시
            start, start_time, prev_frame, prev_label = f, t, f, lab
            continue
        # 라벨 변경 경계
        if lab != prev_label:
            segs.append({"start": start, "end": f, "label": prev_label, "note": f"시간 표현: {start_time} ~ {t}"})
            start, start_time, prev_label = f, t, lab
        prev_frame = f

    segs.append({"start": start, "end": prev_frame + 1, "label": prev_label, "note": f"시간 표현: {start_time} ~ {t}"})
    return segs

def convert_csv_to_segments_json(csv_path, out_json_path, fps=30):
    df = pd.read_csv(csv_path)
    df = df[["frame", "time_sec", "drowsiness_level", "drowsiness_label"]]

    segments = make_segments(df, "frame", "drowsiness_level", "time_sec", insert_gaps=True)
    num_frames = int(df["frame"].max()) + 1

    friendly = {0:"정상", 1:"의심", 2:"주의", 3:"위험", -1:"무시"}
    class_map = [{"id": int(i), "name": friendly.get(int(i))} for i in friendly]

    payload = {
        "schema_version": "1.0",
        "dataset": {
            "id": os.path.splitext(os.path.basename(csv_path))[0],
            "source": csv_path,
            "fps": fps,
            "num_frames": num_frames,
            "timebase": "frame"
        },
        "segments": segments,
        "class_map": class_map,
        "meta": {
            "annotator": "auto-convert + human-verified",
            "created_at": pd.Timestamp.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
            "comments": "half-open [start,end) rule; gaps labeled -1"
        }
    }
    with open(out_json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

In [13]:
for video_path, video_name, csv_path, csv_name in zip(video_file_paths, video_file_names, csv_file_paths, csv_file_names):
    cap = cv2.VideoCapture(video_path)  

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 1e-2: fps = FPS_FALLBACK
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.get(cv2.CAP_PROP_FRAME_COUNT)>0 else None

    convert_csv_to_segments_json(csv_path=csv_path, 
                                 out_json_path='./output/segment/'+csv_name.replace('.csv', '.json'),
                                 fps=fps)

# 튜닝 포인트

* 윈도우 길이: PERCLOS_WIN_SEC(기본 30초), RATE_WIN_SEC(기본 60초). 짧게 하면 반응이 빨라지지만 변동성이 커집니다.

* 임계값: PERCLOS_T*, YAWN_T*, BLINK_DUR_T*, LONG_EC_T*를 데이터에 맞게 조정하세요.
(예: 밤 운전 데이터에서는 PERCLOS 경계가 전체적으로 높게 나올 수 있음)

* 성능 최적화: 실시간이 필요하면
  * refine_landmarks=False
  * 입력 해상도 다운스케일 후 좌표 매핑
  * Hands는 하품 후보일 때만 실행(조건부)

*  프레임 스킵/검출 주기 적용(앞서 드린 DETECT_EVERY_N 패턴)

* 신호 품질 & 전처리
  * 해상도/ROI 다운스케일: FaceMesh는 480p 또는 얼굴 ROI 기준 긴 변 256~320px로 제한 → 연산↓, 추적 안정↑
  * 조명/노이즈: 자동 노출 고정(가능하면), 가우시안(3×3)·샤프닝 약간 → 랜드마크 떨림↓
  * 안경/마스크 케이스: EAR 대신 눈꺼풀 거리의 로버스트 비율(눈 구석 점 고정)로 대체 옵션 마련

* 피처 튜닝(EAR/MAR 외)
  * 고급 PERCLOS: EAR 임계치 하나가 아닌 적응형(중간값±IQR) 임계치로 per-person 보정
  * Blink 품질: 닫힘-열림 모두 관측될 때만 이벤트 확정(+최소/최대 지속시간 클램프)
  * Yawn 강화: MAR + 입 넓이/세로 비율 + 하악(턱) 이동량(하관 랜드마크 평균 변화) 결합
  * Head pose 보조: 고개 떨굼(pitch 상승) 지속 시 drowsy 단계 가중치↑ (PnP로 간단 추정)

* 의사결정 로직(안정화)
  * 히스테리시스: 졸림 단계는 상승·하강 임계 분리(예: 올라갈 때 PERCLOS 0.40, 내려갈 때 0.35).
  * 지속시간 게이팅: 레벨 변동은 연속 T초(예: 5~8s) 유지 시 확정, 아니면 보류.
  * 가중 융합: PERCLOS(0.5)·long closure(0.3)·yawn rate(0.2)처럼 가중 합 → 최종 점수→단계 매핑.
  * 결측 처리: 얼굴 미검출 시 마지막 유효값 유지(최대 2~3s) 후 ‘unknown’ 표기, 레벨은 완만히 감소.

* 윈도우 & 파라미터
  * 적응 윈도우: RATE_WIN_SEC 60s 고정 대신 30–90s 가변(최근 변동성 낮으면 길게).
  * 초기 캘리브: 2s는 짧음 → 5–10s로 늘리고, 추후 지수 이동 평균으로 천천히 업데이트.
  * 개인화: 사용자별 초기 세션에서 EAR/MAR 분포 저장→다음 세션에 로드(프로필별 임계).

* 성능 최적화(실시간성)
  * 조건부 Hands: (MAR>임계 & 6~12프레임 지속)일 때만 실행.
  * DETECT_EVERY_N: FaceDetection 주기 10~15, FaceMesh는 추적 위주.
  * 드로잉 최소화: 텍스트/간단 바만 표시, 전체 랜드마크 드로잉은 디버그 전용.
  * I/O 최적화: CSV는 주기적 flush(예: 2s마다), VideoWriter는 빠른 코덱(preset 빠르게).

* 강건성 & 모니터링
  * 품질 스코어: 랜드마크 신뢰(또는 추론 실패율) 로깅 → 나중에 오탐 구간 분석.
  * 워치독: 연속 미검출 N초면 “카메라 위치 조정” 경고 프레임 생성.
  * 숫자 안정성: NaN/Inf 체크, 분모 0 보호, outlier 클리핑(예: EAR/MAR z-score>4 제거).

* 후처리(부드러운 출력)
  * 레벨 스무딩: 최종 단계에 모드 필터(3~5프레임) 또는 EMA(α=0.2) 적용 → 점멸 방지.
  * 타임라인 바: 프레임 하단에 색 막대로 0~3단계 표시(검수 용이).

* 평가/튜닝 워크플로
  * 라벨 대비 지표: 프레임 F1 + 경계 오차(초) + 레벨 전이 지연(ms) 기록.
  * 그리드 서치: PERCLOS/Yawn/EC 임계를 소범위 격자 탐색→베스트 조합 고정.
  * 활성학습: 모델이 불확실(지표 경계 근처) 구간만 표본 추출해 수동 검수→업데이트.